# Data Science Practical Exam

In [12]:
import dask.array as np
import dask.dataframe as pd
%matplotlib inline
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow as tf 

In [10]:
from IPython.core.display import display,HTML 
display(HTML("<img src='images/exam.jpg' height=500, width=800 style='transform:rotate(180deg);' /> "))

## Q-1: Multiple Regression 

For this question, the task is to apply multiple linear regression on the Boston Housing dataset available on the UCI Machine Learning Repository.

Data is presented in .data format. 

### Before Loading Data in Pandas:

- The data should first be extracted into a .txt format. 
- Using sed within Terminal, comma delimiters need to be added to the data. 
- Labels should be added from the .names as the first line of the txt file. 
- A file conversion needs to be done from the txt file to a csv file. 


In [14]:
ls data/

housing_data.txt  iris_data.csv


In [20]:
house_dat = open('data/housing_data.txt', "w")

lineArr = [i for i in house_dat.readlines()] 

for line in lineArr:
    delim_string = ','.join(line) 
    
  

 0.00632  18.00   2.310  0  0.5380  6.5750  65.20  4.0900   1  296.0  15.30 396.90   4.98  24.00

